In [1]:
import pandas as pd 
import numpy as np 
import sys
import asyncio
sys.path.append('../scripts')

from telegram_scraper import scrape_telegram_channels, download_images_from_channels

In [ ]:
# Load all secretes 

from config import API_HOST, API_PORT, POSTGRES_USER

print(f"Connecting to DB as user: {POSTGRES_USER}")


## Data Scraping and Collection (Extract & Load)
Extract relevant text, image, price, product name, and timestamp data from public Telegram channels and save it to your raw data zone (e.g., JSON or Parquet files).

### Scraping Telegram data using json format

In [3]:
# Run the async scraper function from notebook
await scrape_telegram_channels()

🔍 Scraping messages from: https://t.me/Chemed123


ConnectionError: Cannot send requests while disconnected

### Scraping Telegram image data 
Image Scraping from Telegram Channels

In [2]:
await download_images_from_channels()

🖼️  Downloading images from: https://t.me/Chemed123


ConnectionError: Cannot send requests while disconnected